In [16]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

### Treate DateTime

In [17]:
def transform_iso_in_datetime(value):
    return np.nan if type(value) == float else value.replace('ISODate("', '').replace('")', '')

### Calculate SLA

In [18]:
def calculate_sla(df_int, df):

    sla_list = []

    for index, row in df_int.iterrows():
        client = row['customerCode']
        date = row['currentDate']
        month = date.month
        year = date.year
        new_df = df[(df['customerCode'] == client) & (df['currentDate'] >= "{}-{}-01".format(year, month)) & (df['currentDate'] <= date)]
        sla = new_df.onTimeSolution.sum() / len(new_df.onTimeSolution)
        sla_list.append(0 if np.isnan(sla) else sla)
    
    return sla_list

### Build network callbacks

In [19]:
def build_callbacks():
    """
    Method to build the callbacks for fit the model

    Returns
    -------
        :return: A list of generators with network attributes
        :rtype: list

    """

    early_stop = EarlyStopping(monitor='val_loss',
                               min_delta=0.001,
                               patience=3,
                               mode='min',
                               verbose=1)

    reduce_on_plateau = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.3,
                                          patience=2,
                                          verbose=1,
                                          mode='min',
                                          min_delta=0.001,
                                          cooldown=0,
                                          min_lr=0
                                          )
    
    return [early_stop, reduce_on_plateau]

### Create sets for neural network (with look back)

In [20]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

### Calculate MAPE

In [1]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100